In [ ]:
HTML(read(open("style.css"), String))

# Iterative Deepening

ToDo: Theory add

In [ ]:
using Pkg
# Pkg.add("Chess")
using Chess
using Random
# Pkg.add("DataStructures")
using DataStructures
# Pkg.add("NBInclude")
using NBInclude
using BenchmarkTools

In [ ]:
@nbinclude("EvaluatePosition.ipynb")

In [ ]:
@nbinclude("Memoization.ipynb")

In [ ]:
@nbinclude("QuiescenceSearch.ipynb")

## maxValue

The `maxValue` function takes in 3 arguments and 2 optional arguments.
1. `State` is a chess `state` of type `Board`
1. `score` is the static centipawn evaluation of the `state`
1. `depth` is the number of halfmoves the engine should analyze before terminating 
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process
        
The function returns the maximal centipawn evaluation of the current position for the player playing white where both players have played the optimal moves according to the algorithm and terminating after the given depth. All possible moves are sorted by a `PriorityQueue` using the evaluation of the position. Good moves will be prioritized which will increase the chance of pruning paths. 

In [ ]:
function maxValue(aBoard::AdvBoard, depth::Int64,
                  cache::Dict{UInt64, Tuple{String, Int64, Int64}}, alpha::Int64=-200000, beta::Int64=200000, flagQuiesce::Bool=false)::Int64
    if isterminal(aBoard.state) 
        return terminal_evaluation(aBoard)
    end
    if depth == 0
        #return score
        # 
        if flagQuiesce
            return evaluate(aBoard, quiesceMax, 0, cache, alpha, beta)
        else
            return quiesceSee(aBoard.state, aBoard.score, alpha, beta)
        end
    end
    value = alpha
    queue = PriorityQueue{Move, Int64}()
    for move in moves(aBoard.state)
        undoinfo = domove!(aBoard.state, move)
        val = value_cache(aBoard.hash, depth-1, cache)
        undomove!(aBoard.state, undoinfo)
        if val == nothing
            val = -200000
        end
        enqueue!(queue, move, -val)
    end
    while !isempty(queue)
        move = peek(queue)[1]
        undo = domoveAdv!(aBoard, move)
        value = max(value, evaluate(aBoard, minValue,  depth - 1, cache, value, beta, flagQuiesce))
        undomoveAdv!(aBoard, undo)
        if value >= beta
            return value
        end
        delete!(queue, move)
    end
    return value
end

## minValue

The `minValue` function takes in 3 arguments and 2 optional arguments.
1. `State` is a chess `state` of type `Board`
1. `score` is the static centipawn evaluation of the `state`
1. `depth` is the number of halfmoves the engine should analyze before terminating 
1. `alpha` is optional and is default to -Infinity. Alpha is a minimal value that has been calculated during the recursive process
1. `beta`  is optional and is default to Infinity . Beta is a maximal value that has been calculated during the recursive process
        
The function returns the maximal centipawn evaluation of the current position for the player playing white where both players have played the optimal moves according to the algorithm and terminating after the given depth. All possible moves are sorted by a `PriorityQueue` using the evaluation of the position. Good moves will be prioritized which will increase the chance of pruning paths. 

In [ ]:
function minValue(aBoard::AdvBoard, depth::Int64,
                  cache::Dict{UInt64, Tuple{String, Int64, Int64}}, alpha, beta, flagQuiesce::Bool=false)::Int64
    if isterminal(aBoard.state)
        return terminal_evaluation(aBoard)
    end
    if depth == 0
        # return score
        # return quiesceSee(State, score, -beta, -alpha)
        if flagQuiesce
            return evaluate(aBoard, quiesceMin, 0, cache, alpha, beta)
        else
            return quiesceSee(aBoard.state, aBoard.score, alpha, beta)
        end
    end
    value = beta
    queue = PriorityQueue{Move, Int64}()
    for move in moves(aBoard.state)
        undoinfo = domove!(aBoard.state, move)
        val = value_cache(aBoard.hash, depth-1, cache)
        undomove!(aBoard.state, undoinfo)
        if val == nothing
            val = 200000
        end
        enqueue!(queue, move, val)
    end
    while !isempty(queue)
        move = peek(queue)[1]
        Undo = domoveAdv!(aBoard, move)
        value = min(value, evaluate(aBoard, maxValue, depth - 1, cache, alpha, value, flagQuiesce))
        undomoveAdv!(aBoard, Undo)
        if value <= alpha
            return value
        end
        delete!(queue, move)
    end
    return value
end

The function `value_cache` is a helping function for the `minValue` and `maxValue` function. It takes in 2 arguments:
1. `State` is a chess `state` of type `Board`
1. `depth` is the number of halfmoves the engine should analyze before terminating

The function looks into the Cache and returns any previously saved values for this position. This information is used to sort good moves inside of the PriorityQueue.
The function returns the value for this `state` if the `state` is in the Cache and has a sufficient pre-calculated depth. If the cache does not have an entry for this `state` or entry does not have sufficient depth the function will return `nothing`.

In [ ]:
function value_cache(hash::UInt64, depth::Int64, cache::Dict{UInt64, Tuple{String, Int64, Int64}})
    if hash in keys(cache)
        _, value, d = cache[hash]
        if d >= depth
            return value
        end
    end
    # new move or no entry with enough depth
    return nothing
end

### Function: pd_evaluate
This function performs iterative deepening search on a given state of the board using the evaluation function f and caching the results using the dictionary cache.

Arguments:
1. `State::Board:` The current state of the board.
1. `f::Function:` The evaluation function to use.
1. `score::Int64:` The initial score of the board.
1. `hash::UInt64:` The hash value of the board.
1. `depth::Int64:` The maximum depth to search.
1. `cache::Dict{UInt64, Tuple{String, Int64, Int64}}`: The dictionary to cache the results.

Returns:
1. `bestVal`: The best value found in the search.
1. `depth`: The depth at which the best value was found.

In [ ]:
function pd_evaluate(aBoard::AdvBoard, f::Function, depth::Int64, 
                     cache::Dict{UInt64, Tuple{String, Int64, Int64}})
    bestVal = aBoard.score
    # println("Boards score ", score)
    # start time
    starttime = time()
    d = 1
    alpha = -200000
    beta = 200000
    while d <= depth
        flagQuiesce = d == depth ? true : false
        bestVal = evaluate(aBoard, f,d, cache,alpha, beta, flagQuiesce)
        if abs(bestVal) == 100000
            return bestVal, d
        end
        # Check if is d equals depth and the difference between startime and currenttime increases depth 
        println("Depth is ", d, " and time needed is ", time() - starttime)
        if depth == d && time() - starttime < 10
            depth +=1 
        end
        d +=1
    end
    return bestVal, depth
end

### Function: iterativeDeepening
This function performs iterative deepening search on the given State and returns the best move found.

Arguments:
1. `State::Board:` A chess board in the current state.
1. `score::Int64:` The current score of the board.
1. `hash::UInt64:` The current hash value of the board.
1. `depth::Int64:` The maximum depth to search.
1. `cache::Dict{UInt64, Tuple{String, Int64, Int64}}`: TThe cache dictionary to store the evaluated board states. It is optional and initialized by initCache() function by default.

Returns:
1. `bestVal`: The best score found by the iterative deepening search.
1. `BestMove`: The best move found by the iterative deepening search.

In [ ]:
function iterativeDeepening(aBoard::AdvBoard,  depth::Int64, 
                            cache::Dict{UInt64, Tuple{String, Int64, Int64}} = initCache())
    side = sidetomove(aBoard.state)
    bestVal, depth = (side == WHITE) ? pd_evaluate(aBoard, maxValue, depth, cache) : 
                                       pd_evaluate(aBoard, minValue, depth, cache)
    # println("BestValue $(bestVal)")
    next_moves = moves(aBoard.state)

    BestMoves::Array{Move} = [move for move in next_moves if zobrist_hash(aBoard.state, aBoard.hash, move) in keys(cache) &&
                                                           cache[zobrist_hash(aBoard.state, aBoard.hash, move)][1] == "=" &&
                                                           cache[zobrist_hash(aBoard.state, aBoard.hash, move)][2] == bestVal]
    
    BestMove::Move = rand(BestMoves)
    return bestVal, BestMove
end